In [ ]:
# importy podstawowe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

plt.style.use('default')
sns.set_palette("husl")

: 

# Analiza danych Titanic 

## Krok 1: Wczytanie danych


In [ ]:
# w colabie - wrzuć plik titanic.csv do files
# albo użyj tego linku:
# df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

# lokalnie:
# base_path = Path()
# df = pd.read_csv(base_path / 'titanic.csv')

# colab:
from google.colab import files
uploaded = files.upload()  # wybierz titanic.csv
df = pd.read_csv('titanic.csv')

print(f"dane: {df.shape}")
df.head()

In [ ]:
print(df.dtypes)


braki = df.isnull().sum()
print(braki)


df.describe()

## Krok 2: Posprzątamy trochę te dane

=mamy trochę braków w danych, szczególnie w Age i Cabin.


In [ ]:
df_cleaned = df.copy()

# procenty braków
missing_percent = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100
print(missing_percent.round(2))

# cabin 77% braków - wywalić
# + passengerId, name, ticket - niepotrzebne

columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

print(f"zostało {df_cleaned.shape[1]} kolumn")
df_cleaned.head()

In [ ]:
# braki po usunięciu
print(df_cleaned.isnull().sum())

# age - mediana
median_age = df_cleaned['Age'].median()
df_cleaned['Age'].fillna(median_age, inplace=True)

# embarked - najczęstszy
most_common_embarked = df_cleaned['Embarked'].mode()[0]
df_cleaned['Embarked'].fillna(most_common_embarked, inplace=True)

print("\npo uzupełnieniu:")
print(df_cleaned.isnull().sum())

## Krok 3: Eksploracja danych 

- Jak wyglądają rozkłady różnych zmiennych
- Czy są jakieś oczywiste zależności z przeżyciem
- Czy są jakieś dziwne wartości 



In [ ]:
# podstawowe statystyki
print(df_cleaned.describe())

print("="*40)
print(df_cleaned.describe(include=['object']))

In [ ]:
# ilu przeżyło?
plt.figure(figsize=(8, 6))
survival_counts = df_cleaned['Survived'].value_counts()
plt.pie(survival_counts.values, labels=['Nie przeżył', 'Przeżył'], autopct='%1.1f%%', 
        colors=['lightcoral', 'lightblue'])
plt.title('Przeżycie')
plt.show()

print(f"przeżyło: {survival_counts[1]} ({survival_counts[1]/len(df_cleaned)*100:.1f}%)")
print(f"nie przeżyło: {survival_counts[0]} ({survival_counts[0]/len(df_cleaned)*100:.1f}%)")

In [ ]:
# rozkłady
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# wiek
axes[0, 0].hist(df_cleaned['Age'], bins=30, color='skyblue', alpha=0.7)
axes[0, 0].set_title('Wiek')

# cena
axes[0, 1].hist(df_cleaned['Fare'], bins=30, color='lightgreen', alpha=0.7)
axes[0, 1].set_title('Cena biletu')
axes[0, 1].set_yscale('log')

# klasa
class_counts = df_cleaned['Pclass'].value_counts().sort_index()
axes[1, 0].bar(class_counts.index, class_counts.values, color=['gold', 'silver', 'brown'])
axes[1, 0].set_title('Klasa')

# płeć
sex_counts = df_cleaned['Sex'].value_counts()
axes[1, 1].bar(sex_counts.index, sex_counts.values, color=['lightblue', 'pink'])
axes[1, 1].set_title('Płeć')

plt.tight_layout()
plt.show()

In [ ]:
# płeć vs przeżycie
survival_by_sex = pd.crosstab(df_cleaned['Sex'], df_cleaned['Survived'], margins=True)
print(survival_by_sex)

female_survival = df_cleaned[df_cleaned['Sex'] == 'female']['Survived'].mean() * 100
male_survival = df_cleaned[df_cleaned['Sex'] == 'male']['Survived'].mean() * 100

print(f"kobiety: {female_survival:.1f}%")
print(f"mężczyźni: {male_survival:.1f}%")
print("ogromna różnica!")

In [ ]:
# wykresy płeć
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

survival_by_sex_plot = df_cleaned.groupby(['Sex', 'Survived']).size().unstack()
survival_by_sex_plot.plot(kind='bar', ax=axes[0], color=['lightcoral', 'lightblue'])
axes[0].set_title('Przeżycie wg płci')
axes[0].legend(['Nie', 'Tak'])
axes[0].tick_params(axis='x', rotation=0)

survival_rates = df_cleaned.groupby('Sex')['Survived'].mean() * 100
survival_rates.plot(kind='bar', ax=axes[1], color=['lightblue', 'pink'])
axes[1].set_title('% przeżycia')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# klasa vs przeżycie
survival_by_class = pd.crosstab(df_cleaned['Pclass'], df_cleaned['Survived'], margins=True)
print(survival_by_class)

class_survival = df_cleaned.groupby('Pclass')['Survived'].mean() * 100
print(f"1. klasa: {class_survival[1]:.1f}%")
print(f"2. klasa: {class_survival[2]:.1f}%")  
print(f"3. klasa: {class_survival[3]:.1f}%")
print("im wyższa klasa tym lepiej")

In [ ]:
# wiek vs przeżycie
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
survived_ages = df_cleaned[df_cleaned['Survived'] == 1]['Age']
not_survived_ages = df_cleaned[df_cleaned['Survived'] == 0]['Age']

plt.hist([not_survived_ages, survived_ages], bins=30, alpha=0.7, 
         label=['Nie', 'Tak'], color=['lightcoral', 'lightblue'])
plt.xlabel('Wiek')
plt.title('Wiek vs przeżycie')
plt.legend()

plt.subplot(1, 2, 2)
df_cleaned.boxplot(column='Age', by='Survived', ax=plt.gca())
plt.title('Wiek boxplot')

plt.tight_layout()
plt.show()

print(f"średni wiek przeżyłych: {survived_ages.mean():.1f}")
print(f"średni wiek nie przeżyłych: {not_survived_ages.mean():.1f}")
print("przeżyli trochę młodsi")

In [ ]:
# outliery
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].boxplot(df_cleaned['Age'])
axes[0].set_title('Wiek - outliery?')

axes[1].boxplot(df_cleaned['Fare'])
axes[1].set_title('Cena - outliery?')

plt.tight_layout()
plt.show()

print("najdroższe:")
expensive_tickets = df_cleaned.nlargest(5, 'Fare')[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]
print(expensive_tickets)

print("najstarsi:")
oldest_passengers = df_cleaned.nlargest(5, 'Age')[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]
print(oldest_passengers)

## nowe cechy które mogą być przydatne

1. **Rodzina** - może podróżowanie z rodziną wpływało na szanse?
2. **Grupy wiekowe** - może lepiej podzielić wiek na kategorie?
3. **Czy ktoś był sam** - może samotni mieli inne szanse?


In [ ]:
# nowe cechy
df_final = df_cleaned.copy()

# rozmiar rodziny
df_final['FamilySize'] = df_final['SibSp'] + df_final['Parch'] + 1

# czy sam
df_final['IsAlone'] = (df_final['FamilySize'] == 1).astype(int)

# grupy wiekowe
df_final['AgeGroup'] = pd.cut(df_final['Age'], 
                             bins=[0, 12, 18, 35, 60, 100], 
                             labels=['Dziecko', 'Nastolatek', 'Dorosły', 'ŚrednioWiek', 'Senior'])

print(f"rodzina: {df_final['FamilySize'].min()}-{df_final['FamilySize'].max()}")
print(f"samotnych: {df_final['IsAlone'].sum()}")
print(df_final['AgeGroup'].value_counts())

# czy wpływa na przeżycie?
family_survival = df_final.groupby('FamilySize')['Survived'].mean()
print(family_survival.round(3))

In [ ]:
# wykresy nowych cech
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

family_survival.plot(kind='bar', ax=axes[0, 0], color='lightgreen')
axes[0, 0].set_title('Rodzina vs przeżycie')
axes[0, 0].tick_params(axis='x', rotation=0)

alone_survival = df_final.groupby('IsAlone')['Survived'].mean()
alone_survival.plot(kind='bar', ax=axes[0, 1], color='orange')
axes[0, 1].set_title('Samotni vs rodzina')
axes[0, 1].tick_params(axis='x', rotation=0)

age_group_survival = df_final.groupby('AgeGroup')['Survived'].mean()
age_group_survival.plot(kind='bar', ax=axes[1, 0], color='purple')
axes[1, 0].set_title('Wiek vs przeżycie')
axes[1, 0].tick_params(axis='x', rotation=45)

survival_sex_class = df_final.groupby(['Sex', 'Pclass'])['Survived'].mean().unstack()
survival_sex_class.plot(kind='bar', ax=axes[1, 1])
axes[1, 1].set_title('Płeć + klasa')
axes[1, 1].legend(title='Klasa', labels=['1', '2', '3'])
axes[1, 1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## Krok 5: Przygotowanie finalnych danych

Teraz muszę przygotować dane do dalszej analizy. Muszę zakodować zmienne kategorialyczne żeby algorytmy mogły z nimi pracować.

In [ ]:
# kodowanie

# płeć: male=0, female=1
df_final['Sex'] = df_final['Sex'].map({'male': 0, 'female': 1})

# dummy variables
df_final = pd.get_dummies(df_final, columns=['Embarked'], prefix='Embarked')
df_final = pd.get_dummies(df_final, columns=['AgeGroup'], prefix='Age')

print(f"kształt: {df_final.shape}")
print(f"braki: {df_final.isnull().sum().sum()}")

df_final.head()

## Podsumowanie 

### Najważniejsze wnioski:

1. **Płeć** - kobiety: 74%, mężczyźni: 19%. Zasada "kobiety i dzieci pierwsze" działała.

2. **Klasa** - 1. klasa: 63%, 2. klasa: 47%, 3. klasa: 24%. Pieniądze miały znaczenie.

3. **Wiek** - małe różnice, dzieci trochę lepiej.

4. **Rodzina** - samotni gorzej, ale duże rodziny też źle.

5. **Kombinacje** - najlepiej: kobiety 1. klasa (97%), najgorzej: mężczyźni 3. klasa (13%).

### Dane gotowe:
- brak braków
- wszystko zakodowane 
- nowe cechy dodane